# **Step 0：ライブラリの読み込み**

In [ ]:
!pip install rdkit

In [ ]:
import rdkit
from rdkit import rdBase, Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors, PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem import AllChem, Draw
from copy import deepcopy
import numpy as np
import pandas as pd
from numpy import vectorize as vec
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
print(rdBase.rdkitVersion)

# **Step 1：SDFファイルの読み込み**

In [ ]:
# MDLフォーマットのファイル(nr-ar.sdf)を読み込む
# 読み込めない化合物もある（本当は事前に確認して削っておくべき）
# nr-ar.sdfはこちらから取得できます。 https://github.com/hatanaka-lab/Getting_started_with_MI/blob/main/data/nr-ar.sdf
# データの元ネタはこちら：https://tripod.nih.gov/tox21/challenge/
df_mols=PandasTools.LoadSDF('nr-ar.sdf',smilesName='SMILES',molColName='Mol_Object',includeFingerprints=True)

In [ ]:
#molsの中身を確認①項目
df_mols.columns

In [ ]:
#molsの中身を確認②データ数
df_mols.shape

In [ ]:
#molsの中身を確認③active/inactiveの化合物数
print("Num of active mols: ", list(df_mols.Active).count('1'))
print("Num of inactive mols: ", list(df_mols.Active).count('0'))

In [ ]:
#molsの中身を確認④
df_mols

# **Step2：RDKitを用いた特徴量(記述子)への変換**

In [ ]:
#rdkitの機能で算出可能な記述子の一覧
names = [x[0] for x in Descriptors._descList]
print("Number of descriptors in the rdkit: ", len(names))
np.array(names)

In [ ]:
# 本当は上記の記述子を全部使いたいけれど…
# ここでは練習のため、一部を(適当に)pick up
df_mols_feat = deepcopy(df_mols)
for desc in ['TPSA','MaxPartialCharge','SlogP_VSA1','EState_VSA1','SMR_VSA1','MolLogP','MolMR','BalabanJ','HallKierAlpha','Kappa1','Kappa2','Kappa3','RingCount','NumHAcceptors','NumHDonors']:
    exec("df_mols_feat[desc]=vec(Descriptors.{})(df_mols_feat['Mol_Object'])".format(desc))
print("shape of data : {}".format(df_mols_feat.shape))
df_mols_feat

# **Step3：欠損データの確認**

In [ ]:
# 特徴量の中に欠損データがある場合は"True"と表示される → 説明変数から外す
print(df_mols_feat.isnull().any())

In [16]:
# 欠損データを含む特徴量(MaxPartialCharge)や、数値以外のデータを含む列(Formula, FW,…,Molecule)、目的変数(Active)を外して、再度説明変数を定義
df_mols_feat=df_mols_feat.drop(["MaxPartialCharge", "Formula", "FW", "DSSTox_CID", "ID", "SMILES", "Mol_Object", "Active"], axis=1)

In [ ]:
# 再度特徴量に欠損データがないか確認（確認方法①）
print(df_mols_feat.isnull().any())

In [ ]:
# 再度特徴量に欠損データがないか確認（確認方法②）
print(df_mols_feat.isnull().sum())

In [ ]:
# 最終的に残った特徴量を表示
df_mols_feat

In [20]:
#　説明変数を標準化する
#　標準化された値 = (元の値－平均値)/標準偏差
#  ここではRandom Forest Classiferを用いるので標準化しなくても良い
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(df_mols_feat)

# **Step4：データを観察**

In [ ]:
# データの分布を観察する：その1（説明変数間の散布図）
def set_color(L):
    O = []
    for l in L:
        if l == '1':
            O.append("red")
        else:
            O.append("palegreen")
    return O

axes = pd.plotting.scatter_matrix(df_mols_feat, figsize=(12,12), hist_kwds={'bins':10},
                           marker='.', s=15, alpha=.5, c=set_color(df_mols.Active))
for ax in axes.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')
plt.show()

In [ ]:
# データの分布を観察する：その2（相関行列）
correlation_coefficients = df_mols_feat.corr()  # 相関行列の計算
# 相関行列のヒートマップ (相関係数の値あり)
plt.rcParams['font.size'] = 9
plt.figure(figsize=(12, 10))  # この段階で画像のサイズを指定する
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, annot=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.show()

# **Step5：データを分割（交差検証用）**

In [23]:
# データを訓練用・テスト用に分割
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_mols_feat, df_mols.Active, train_size=0.75, test_size=0.25, random_state=10)

In [ ]:
# 分割したデータの個数を確認
print("Training Data -------------")
print(" Number of   active molecules:  ", list(y_train).count('1'))
print(" Number of inactive molecules:  ", list(y_train).count('0'))
print("Test Data -----------------")
print(" Number of   active molecules:  ", list(y_test).count('1'))
print(" Number of inactive molecules:  ", list(y_test).count('0'))

## **Step6：ハイパーパラメタの決定**

In [ ]:
# Random Forest Classifier(RFC)用にハイパーパラメタを決める
# RFCの各パラメータの意味はこちらを参照
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
list_param = []
list_score = []
for ne in range(50,250,50):
  for nd in [5, 10, 25, 50, 100]:
    model = RandomForestClassifier(n_estimators=ne, max_depth=nd, random_state=111)
    cv5_score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()  #訓練用データ(train)を用いて5-fold CV
    print("num_trees=",ne,", max_depth=",nd," , Accuracy=",cv5_score)
    list_param.append([ne,nd])
    list_score.append(cv5_score)
max_index = np.argmax(list_score)
print("")
print("-----Best parameters-----")
print("num_trees=",list_param[max_index][0], "max_depth=",list_param[max_index][1],"Accuracy=",list_score[max_index])

# **Step7：機械学習実行**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=list_param[max_index][0], max_depth=list_param[max_index][1], random_state=111)
model.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:     {:.3f}".format(model.score(X_test,  y_test)))

In [ ]:
from sklearn.metrics import auc, accuracy_score, f1_score, confusion_matrix, log_loss, roc_auc_score
plt.figure(figsize=(3,2))
cm = confusion_matrix(y_test, model.predict(X_test))
ax = plt.subplot()
sns.set()
sns.heatmap(cm, annot=True, ax=ax, cmap="Blues", fmt="g");

# Labels, title and ticks
label_font = {'size':'14'}  # Adjust to fit
ax.set_xlabel('Predict', fontdict=label_font);
ax.set_ylabel('True',  fontdict=label_font);

title_font = {'size':'14'}  # Adjust to fit
ax.set_title('Confusion Matrix', fontdict=title_font);

#ax.tick_params(axis='both', which='major', labelsize=10)  # Adjust to fit

(Remind： Active = 1, Inactive = 0)

In [ ]:
# Random Forest回帰モデルの特徴量重要度を解析
# まずは表形式（DataFrame型）で出力
model_importances = pd.DataFrame({"importance": model.feature_importances_}, index=X_train.columns)
# 重要度ごとの降順にして表示
model_importances_sorted = model_importances.sort_values(by="importance", ascending=False)
model_importances_sorted

In [ ]:
# 棒グラフで重要度を図示
fig, ax = plt.subplots(figsize=(6,4))
y_pos = np.arange(model_importances_sorted.shape[0])
list_importances = list(model_importances_sorted["importance"])
ax.barh(y_pos, list_importances, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_yticks(y_pos)
ax.set_yticklabels(model_importances_sorted.index)
label_imp_font = {'size':'11'}
title_imp_font = {'size':'12'}
ax.set_xlabel('Importance', fontdict=label_imp_font)
ax.tick_params(axis='both', which='major', labelsize=11)
ax.yaxis.grid(True)
plt.tight_layout()
plt.show()

# **補足：分子情報がSMILESで与えられている場合**

分子の通し番号(ID), SMILES, 物性値(DATA)をまとめたcsvファイルを読み込みます。

ファイルはこちら：https://github.com/hatanaka-lab/Getting_started_with_MI/blob/main/data/smiles_data.csv

上記のSDFファイル由来のデータと混ざらないように、オブジェクト名をmols_2にしました。分子数が多いので、計算に時間がかかります。csvファイルの行数を適当に減らしてテスト計算してみても良いです。

In [ ]:
# csvファイルの中身をpandasのdata frameに格納
df_mols2 = pd.read_csv("smiles_data.csv")
df_mols2.head(4)

物性値が連続値であるため、回帰モデルを構築することも可能ですが、

ここでは、DATA>6をActive、DATA<=6をInactiveとする二値分類問題として扱います。

In [ ]:
# mol_2に'Active'という列を追加し、Active/Inactiveの情報を書き込む
# やり方①：Active/Inactiveを1/0として書き込む
df_mols2['Active'] = [1 if mol > 6 else 0 for mol in df_mols2['DATA']]

# やり方②：Active/InactiveをTrue/Falseとして書き込む
# df_mols2['Active'] = df_mols2['DATA'] > 6

df_mols2.head(3)

In [32]:
Chem.PandasTools.AddMoleculeColumnToFrame(df_mols2, smilesCol='SMILES', molCol='Mol_Object')

In [ ]:
print("Number of   active molecules:  ", list(df_mols2.Active).count(1))  #Active
print("Number of inactive molecules:  ", list(df_mols2.Active).count(0))  #Inactive

In [ ]:
# df_mols2の中身を確認
df_mols2

In [35]:
#あとは上記と同じようにRdkitを使って記述子に変換
#今回は全部使用する
# molオブジェクトから特徴量への変換を行う計算モジュール(MoleculeDescriptors.MolecularDescriptorCalculator())を用意
target_descriptors = []
for desc in Descriptors.descList:
    target_descriptors.append(desc[0])
descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(target_descriptors)

# 関数を定義
def mol2descriptors(mol):
    return descriptor_calculator.CalcDescriptors(mol, ignore_3D=False)

In [ ]:
list_features = []
for mol in tqdm(df_mols2['Mol_Object']):
# for mol in df_mols2['Mol_Object']: でもOK。tqdm()は計算の進捗を確認するためにつけただけ
    list_features.append(mol2descriptors(mol))
df_mols2_feat = pd.DataFrame(list_features, columns=[col for col in target_descriptors])

In [ ]:
# 欠損値がある場合は特徴量から除外する。
# 欠損値あり＝Trueを見つけたらprintする。
df_isnull = df_mols2_feat.isnull().any()
df_isnull[df_isnull > 0]

In [ ]:
# 欠損値のある特徴量をlist形式で取り出す
list(df_isnull[df_isnull > 0].index)

In [39]:
# 欠損データを含む特徴量を外して、再度説明変数を定義
df_mols2_feat=df_mols2_feat.drop(list(df_isnull[df_isnull > 0].index), axis=1)

In [ ]:
## 単一の値しか持たない特徴量がある場合も除外する。
list_1unique_value = []
for col in df_mols2_feat.columns:
    # ユニークな値の個数をカウント
    n_unique_value = len(np.unique(df_mols2_feat[col].values))
    if n_unique_value <= 1:
        list_1unique_value.append(col)
list_1unique_value

In [41]:
df_mols2_feat=df_mols2_feat.drop(list_1unique_value, axis=1)

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_mols2_feat, df_mols2.Active, test_size=0.2, random_state=123)

In [ ]:
# 機械学習 (ハイパーパラメタ調整は省略)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=111)
model.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:     {:.3f}".format(model.score(X_test,  y_test)))

# **補足２：Morgan fingerprintを特徴量に用いる場合**

ECFP (extended-connnectivity fingerprint)とも言う。

各原子に対して識別子と呼ばれる整数値を割り当て、その原子のまわりの情報や隣接する原子の識別子を用いて、繰り返し更新することで、分子グラフの情報を数値化したもの。

In [44]:
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect

In [ ]:
# 練習のため、一分子のMorgan fingerprintを表示してみる。
mol_test = Chem.MolFromSmiles('CC1=CC(=O)C=CC1=O')
fp = GetMorganFingerprintAsBitVect(mol_test, radius=2, nBits=2048, useFeatures=True)
import torch
print(torch.tensor(fp))  # Morgan fingerprint
print(torch.tensor(fp.GetOnBits()))  # 非ゼロ成分の番号を表示

In [ ]:
# smiles_data.csvの分子(df_mols2に格納済み)のMorgan fingerprintを作成
fps = []
for mol in df_mols2['Mol_Object']:
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048, useFeatures=True)
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    fps.append(arr)
fps = np.array(fps)
fps

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(fps, df_mols2.Active, test_size=0.2, random_state=123)

In [ ]:
# 機械学習 (ハイパーパラメタ調整は省略)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=111)
model.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(model.score(X_train, y_train)))
print("Accuracy on test set:     {:.3f}".format(model.score(X_test,  y_test)))